In [33]:
import pandas as pd
import numpy as np

# # DOMA
# nodes_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Nodes_processed.csv"
# roads_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Roads_processed.csv"
# bridges_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\df_bridges_data.csv"
# survey_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\surveys\Survey.csv"

# ROBOTA
nodes_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\Nodes_processed.csv"
roads_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\Roads_processed.csv"
bridges_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\df_bridges_data.csv"
survey_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\surveys\Survey.csv"

# Load the CSV data into a DataFrame
df_nodes = pd.read_csv(nodes_path, encoding='utf-8', delimiter=',')
df_roads = pd.read_csv(roads_path, encoding='utf-8', delimiter=',')
df_bridges_all = pd.read_csv(bridges_path, encoding='utf-8', delimiter=',')
df_survey = pd.read_csv(survey_path, encoding='utf-8', delimiter=',')

C:\Users\relia\AppData\Local\Temp\ipykernel_8336\2451089063.py:17: DtypeWarning: Columns (5,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nodes = pd.read_csv(nodes_path, encoding='utf-8', delimiter=',')
C:\Users\relia\AppData\Local\Temp\ipykernel_8336\2451089063.py:18: DtypeWarning: Columns (10,11,12,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_roads = pd.read_csv(roads_path, encoding='utf-8', delimiter=',')


In [34]:
# Assuming df_bridges_all is your DataFrame
columns_to_convert = ['Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška', 'Normálna_zaťažiteľnosť',
                      'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t', 'Počet_otvorov',
                      'Dĺžka_nosnej_konštrukcie_m', 'Dĺžka_premostenia', 'Volná_šírka_mosta',
                      'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2', 'Plocha_mosta_m2']

# Convert specified columns to float
df_bridges_all[columns_to_convert] = df_bridges_all[columns_to_convert].apply(
    pd.to_numeric, errors='coerce')

# Convert Rok_postavenia to integer, assign NaN if not possible
df_bridges_all['Rok_postavenia'] = pd.to_numeric(
    df_bridges_all['Rok_postavenia'], errors='coerce')


df_bridges_all.columns

Index(['ID_mosta', 'Trieda_PK', 'Číslo_PK', 'Most_DC', 'Názov_mosta', 'ID_DC',
       'správcovské_číslo', 'Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška',
       'Druh_konštrukcie', 'Material', 'Normálna_zaťažiteľnosť',
       'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t',
       'Spôsob_stanovenia', 'Predmet_premostenia', 'Rok_postavenia',
       'Zaťažovacia_trieda_STN', 'Počet_otvorov', 'Dĺžka_premostenia',
       'Dĺžka_nosnej_konštrukcie_m', 'Volná_šírka_mosta',
       'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2',
       'Plocha_mosta_m2', 'Vlastník_úseku', 'Správca_úseku', 'Okres',
       'Smer_úseku', 'Typ_úseku', 'Počiatočný_uzol', 'Koncový_uzol', 'úsek_ID',
       'úsekové_staničenie_začiatku_DC_m',
       'Kilometrovníkové_staničenie_začiatku_DC_km',
       'Kumulatívne_staničenie_začiatku_DC_km', 'Poradie', 'Kraj', 'n_2012',
       'n_2013', 'n_2014', 'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019',
       'n_2020', 'n_2021', 'n_2022', 'n_2023'],
      

In [35]:
# DOMA
# SC_NR = r"C:\Users\Doktor\Documents\GitHub\Bridges\04_SC_NR\DT\sorted_bridges_paths_NR.xlsx"

# ROBOTA
SC_NR = r"C:\Users\relia\Documents\GitHub\Bridges\04_SC_NR\DT\sorted_bridges_paths_NR.xlsx"

df_SC_NR = pd.read_excel(SC_NR)

In [36]:
def to_string_list(s):
    # Check if s is NaN (missing value)
    if pd.isna(s):
        return []
    # If s is a float or integer, convert it directly to a list with a single element as a string
    elif isinstance(s, (float, int)):
        # Convert to integer first to remove any decimals
        return [str(int(s))]
    # If s is a string, attempt to split it into a list of strings
    elif isinstance(s, str) and s != '':
        # Assume it's a comma-separated list and split accordingly
        return s.split(', ')
    # Return an empty list for any other cases (e.g., empty strings)
    else:
        return []

In [37]:
df_SC_NR['Číslo_sčítacieho_úseku'] = df_SC_NR['Číslo_sčítacieho_úseku'].apply(
    to_string_list)

In [38]:
def get_intenzita(cislo_scitacieho_useku):
    if not cislo_scitacieho_useku:  # Check if the list is empty
        return 0  # Return zero if the list is empty
    else:
        # Find matching 'S' values and return them as a list
        intenzita_list = df_survey[df_survey['ÚSEK'].isin(
            cislo_scitacieho_useku)]['S'].tolist()
        if not intenzita_list:  # If no matches found, return zero
            return 0
        return intenzita_list

In [39]:
def calculate_mean_intenzita(intenzita):
    # Check if intenzita is a list and not empty
    if isinstance(intenzita, list) and intenzita:
        # Calculate mean if there are elements in the list
        return np.mean(intenzita)
    # Return NaN for empty lists or if intenzita is 0
    return np.nan

In [43]:
df_SC_NR['intenzita'] = df_SC_NR['Číslo_sčítacieho_úseku'].apply(
    get_intenzita)

In [46]:
df_SC_NR['intenzita'] = df_SC_NR['intenzita'].apply(calculate_mean_intenzita)

In [57]:
# Sort the DataFrame by the 'intenzita' column in ascending order, treating NaNs as the lowest values
df_SC_NR_sorted = df_SC_NR.sort_values(
    by='intenzita', ascending=False, na_position='first')

df_SC_NR_sorted

,ID_mosta,Názov_mosta,Zemepisná_šírka,Zemepisná_dĺžka,Číslo_sčítacieho_úseku,intenzita
136,M7550,Most cez odvodňovací kanál Trávnik - Kližská N...,47.753535,17.800339,[],NaN
137,M2925,Most cez odvodňovací kanál Veľké Kosihy.,47.751302,17.861275,[],NaN
138,M4666,Most cez prívodný kanál k čerpacej stanici v ...,47.748784,17.875707,[],NaN
145,M6669,Most cez odvodňovací kanál v obci Lipové.,47.839509,17.854950,[],NaN
146,M414,Most cez odvodňovací kanál v obci Veľké Kosihy.,47.753100,17.882176,[],NaN
...,...,...,...,...,...,...
384,M1076,Most cez potok za obcou Jatov,48.116826,18.000753,[85800],324.0
181,M2667,Most cez potok Sikenica - Šalov,47.995816,18.698874,[83307],258.0
182,M1142,Most cez potok Teplica - Šalov,48.002995,18.707463,[83307],258.0
382,M1697,Most cez potok Cergát v Palárikove,48.044312,18.074485,[81658],256.0


In [54]:
df_SC_NR['intenzita'].describe()

count      319.000000
mean      2728.028213
std       2200.335971
min        228.000000
25%       1238.000000
50%       2117.000000
75%       3739.000000
max      13513.000000
Name: intenzita, dtype: float64